# Assignment 1c Notebook: Self-Adaptive Parameter Control
This notebook will guide you throught the second part of this two-part assignment: building an EA with self-adaptive parameter control. If you haven't completed `1c_notebook0.ipynb` yet, progress no further and complete that notebook first!

As usual, be sure to **read all of this notebook** and you can start by executing the next cell.

In [ ]:
# Configure this notebook to automatically reload modules as they're modified
# https://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (12.0, 12.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'

# Note: we're skipping the plotMaze function from the other notebooks, but 
# you're welcome to copy and paste it here if you want to look at mazes in-notebook.

print('The first cell has been executed!')

## Self-Adaptive Paremeter Control
First off, let's clear something up.

**Adaptive paremeter control $\neq$ self-adaptive paremeter control.**

We see multiple students every year who conflate these two techniques and we'd like to take the opportunity to explicitely say that they are different. In *adaptive* parameter control, one might dynamically set mutation rate based on the diversity of the entire population. In *self-adaptive* paremeter control, mutation rate (or some other parameter) is encoded within each individual of the population as an additional chromosome. This chromosome is then used as a parameter that only effects the individual or a small portion of the population (but ideally not the entire population).

The intuition behind self-adaptive parameter control is that there is a correlation between high fitness individuals and the parameter chromosome(s) encoded within them. In other words: if a parameter chromosome produces high-quality solutions, then that parameter chromosome has a greater chance of propagating with the high-quality solution chromosomes. This, in theory, allows evolution to tune itself to some extent.

In this assignment, you'll implement a self-adaptive mutation rate where a child's chance to mutate is determined by the mutation rate chromosome they carry. We'll accomplish this by inheriting class definitions you've already implemented so we can rewrite as little code as possible.

To begin, implement the `randomInitialization` member function in `selfAdaptiveGenotype.py` and test it out in the next cell.

In [ ]:
from selection import *
from snakeeyes import readConfig
from selfAdaptiveGenotype import selfAdaptiveGenotype
from fitness import repair_and_test_map

config = readConfig('./configs/green1c1_config.txt', globalVars=globals(), localVars=locals())

testSolution = selfAdaptiveGenotype()
testSolution.randomInitialization(**config['initialization_kwargs'])

min_parameter = config['initialization_kwargs']['min_parameter']
max_parameter = config['initialization_kwargs']['max_parameter']
print(f'The randomInitialization function did something? {testSolution.parameter is not None}')
print(f"The individual's mutation rate was {testSolution.parameter}")
print(f'mutation rate was within the right range? {min_parameter <= testSolution.parameter <= max_parameter}')

testSolution.fitness, testSolution.log = repair_and_test_map(testSolution.gene, **config['fitness_kwargs'])

game_log_path = 'worldFiles/1cnotebooktest.txt'
with open(game_log_path, 'w') as f:
	[f.write(f'{line}\n') for line in testSolution.log]
    
print(f"The solution's fitness was {testSolution.fitness} and the game log was written to {game_log_path}")
del testSolution # to discourage haphazard copypasta

Assuming `selfAdaptiveGenotype.randomInitialization` is implemented correctly, you should automatically get the `initialization` class method via inheritance without the need for any additional modification. Test your `randomInitialization` at a slightly larger scale in the next cell.

In [ ]:
examplePopulation = selfAdaptiveGenotype.initialization(10, **config['initialization_kwargs'])

print(f'Population size: {len(examplePopulation)}')
print(f'Number of uninitialized individuals: {len([individual.parameter for individual in examplePopulation if individual.parameter is None])}')
print(f'Number of individuals with out of bounds mutation rate: {len([individual.parameter for individual in examplePopulation if not (min_parameter <= individual.parameter <= max_parameter)])}')
del examplePopulation # to discourage haphazard copypasta

## Fitness Evaluation
In the next cell, write a function to evaluate the fitness of an input population. Note that this should be the *vanilla* (non-penalized) fitness evaluation like you implemented in Assignment 1b and not the penalized fitness you implemented in the first 1c notebook. So that you can copy your 1b evaluation function over more easily, we'll use the same parameter interface as the 1b notebook.

In [ ]:
# evaluate the population and assign fitness and logs as described above
def evaluate_population(population, fitness_kwargs):
    pass

In [ ]:
import statistics

examplePopulation = selfAdaptiveGenotype.initialization(10, **config['initialization_kwargs'])

# calling your function to test things out
evaluate_population(examplePopulation, config['fitness_kwargs'])

print(f'Individuals with unassigned fitness: {len([individual.fitness for individual in examplePopulation if individual.fitness is None])}')
print(f'Number of fitness evaluations performed: {len([individual.fitness for individual in examplePopulation if individual.fitness is not None])}')
print(f'Average fitness of population: {statistics.mean([individual.fitness for individual in examplePopulation])}')
maxFitness = max([individual.fitness for individual in examplePopulation])
print(f'Best fitness in population: {maxFitness}')
bestLog = None
for individual in examplePopulation:
    if individual.fitness == maxFitness:
        bestLog = individual.log
        break

print(f'Found log of highest scoring individual? {bestLog is not None}')
with open(game_log_path, 'w') as f:
	[f.write(f'{line}\n') for line in bestLog]
    
print(f"The log of the most fit individual was written to {game_log_path}")
del examplePopulation # to discourage haphazard copypasta

## Recombination
Now that fitness evaluation is up and running, implement the `recombination` method in `selfAdaptiveGenotype.py` and test it in the next cell.

In [ ]:
config = readConfig('./configs/green1c1_config.txt', globalVars=globals(), localVars=locals())

parents = selfAdaptiveGenotype.initialization(2, **config['initialization_kwargs'])
evaluate_population(parents, config['fitness_kwargs'])

parent0 = parents[0]
parent1 = parents[1]

child = parent0.recombine(parent1, **config['recombination_kwargs'])
print(f'The function did something? {child.parameter is not None}')
del parents

## Mutation
Finish the implementation of `selfAdaptiveGenotype.py` by implementing the `mutate` method and testing it in the following cell.

In [ ]:
config = readConfig('./configs/green1c1_config.txt', globalVars=globals(), localVars=locals())
mutant = child.mutate(**config['mutation_kwargs'])
print(f'The function did something? {child.parameter != mutant.parameter}')
del mutant

## Child generation
With the self-adaptive genotype implmented, the EA class only requires minor modifications to use a `generate_children` function that mutates children based on the `parameter` chromosome they contain. To accomplish this, implement the `generate_children` member function in `selfAdaptiveEvolution.py` and test the updated EA class in the next cell.

In [ ]:
from selfAdaptiveEvolution import selfAdaptiveEvolutionPopulation

config = readConfig('./configs/green1c1_config.txt', globalVars=globals(), localVars=locals())

exampleEA = selfAdaptiveEvolutionPopulation(**config['EA_configs'], **config)
evaluate_population(exampleEA.population, config['fitness_kwargs'])
exampleEA.evaluations = len(exampleEA.population)
print(f'Average fitness of population: {statistics.mean([individual.fitness for individual in exampleEA.population])}')
print(f'Best fitness in population: {max([individual.fitness for individual in exampleEA.population])}')
print(f'Number of fitness evaluations: {exampleEA.evaluations}')

children = exampleEA.generate_children()
evaluate_population(children, config['fitness_kwargs'])
exampleEA.evaluations += len(children)
print(f'Average fitness of children: {statistics.mean([individual.fitness for individual in children])}')
print(f'Best fitness of children: {max([individual.fitness for individual in children])}')
print(f'Number of fitness evaluations: {exampleEA.evaluations}')

exampleEA.population += children

exampleEA.survival()
print(f'Average fitness of population: {statistics.mean([individual.fitness for individual in exampleEA.population])}')
print(f'Best fitness in population: {max([individual.fitness for individual in exampleEA.population])}')
del exampleEA

You may have noticed that the EA class automatically used your `selfAdaptiveGenotype` class. That's because the class is passed as an object via the config file to minimize the amount of rewriting your EA!

## Experimentation
Now that you've implemented a full EA with self-adaptive parameter control, it's time to set up experiments! To begin, start out by performing a single run of your EA in the next cell. Note that this will likely be nearly identical to the single run experiment you wrote in the 1b notebook except for the use of a new config file and EA class. Be very careful to use the right config file if you copy and paste!

In [ ]:
number_evaluations = 2000

# You can parse different configuration files here as necessary
config = readConfig('./configs/green1c1_config.txt', globalVars=globals(), localVars=locals())

# implement your EA here


Now, implement code to perform 30 runs of your EA search that each contain 2,000 evaluations. For each generation of each run, log the mean and best fitness of the current population, **the average mutation_rate of the current population**, and well as the number of fitness evaluations performed so far (including the initial population). Also for each run, record the best fitness found during the run.

In [ ]:
number_runs = 30
number_evaluations = 2000

# You can parse different configuration files here as necessary
config = readConfig('./configs/green1c1_config.txt', globalVars=globals(), localVars=locals())

# implement your multi-run experiment here


## Report
Using the data you've collected from your 30 run experiment, average per-generation across all runs to find the average mean and maximum population fitnesses **as well as average mutation_rate** across 30 runs. Using this data, produce a plot that shows the mean and best fitness **as well as mean mutation_rate** per number of fitness evaluations averaged over 30 runs. This is the same plot as Assignment 1b **but with a line for mutation_rate**. Include this in your report along with any statistical analysis or additional requested components from the assignment description. Statistical analysis should consist of a comparison between best per-run fitness values of your 1b and 1c EA searches. Use data generated with the 1b notebook to compare against your 1c EA results.